<a href="https://colab.research.google.com/github/changyuhsin1999/duke_chatbot/blob/main/notebooks/Copy_of_Create_ft_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pinecone-client
!pip install sentence_transformers
!pip install openai==0.28
!pip install transformers

In [3]:
import argparse
import hashlib
import io, re, os
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, PipelineTool
import openai
from pinecone import init, Index, Pinecone, ServerlessSpec
import pinecone
import logging

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
!git clone https://github.com/mille055/duke_chatbot.git

fatal: destination path 'duke_chatbot' already exists and is not an empty directory.


In [5]:
qa_pairs_json = json.load(open('/content/duke_chatbot/data/fine_tune_data.json'))


In [6]:
qa_pairs_json[10]

{'input': 'What is the flexibility of the degree program?',
 'output': 'Duke offers on-campus or online study options. You can start with the summer pre-program online data science and programming boot camp. The course can be finished in as little as 12 months of studying full-time through the accelerated course schedule, or you can stay for a third semester and focus on industry-oriented electives.'}

In [7]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('Pinecone')
OPENAI_API_KEY = userdata.get('OpenAI')


In [8]:
pc = Pinecone(api_key=PINECONE_API_KEY)


# List all indexes in your project
print(pc.list_indexes())


{'indexes': [{'dimension': 1536,
              'host': 'chatbot0411-1r32hfx.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'chatbot0411',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 384,
              'host': 'duke-chatbot-1r32hfx.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'duke-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 384,
              'host': 'newindex-1r32hfx.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'newindex',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'h

In [32]:
import openai

openai.api_key = userdata.get('OpenAI')

openai_embedding_model = 'text-embedding-3-small'

def semantic_search(query):
    source_list = []
    texts = []
    try:
        # Embed the query using OpenAI's embedding API
        response = openai.Embedding.create(
            engine=openai_embedding_model,
            input=query  # Ensuring the input is in the correct format
        )
        # Extract the embedding
        query_embedding = response['data'][0]['embedding']

        pc = Pinecone(api_key=PINECONE_API_KEY)

        # Get the index
        index_name = "dukechatbot0411"
        index = pc.Index(index_name)


        # Query the vector index, make sure your index is properly set up to handle this
        results = index.query(vector=query_embedding, top_k=5, include_metadata=True)
        matches = results["matches"]

        for match in matches:
            source_list.append(match['metadata']['source'])
            texts.append(match['metadata']['text'])

        return texts, source_list

    except Exception as e:
        print(f"Error during semantic search: {e}")
        return [], []


In [33]:
query = "What does AIPI 540 covers?"
texts, sources = semantic_search(query)
print(texts, sources)

['ugh the AI MEng program. AIPI Departmental Electives AIPI 530: Optimization in Practice Optimization is the ultimate skill in artificial intelligence and prescriptive analytics allowing practitioners to generate the best actionable solutions for business needs. ', ' traditional industries. Product development Core AIPI 560: Legal, Societal & Ethical Implications of AI Deploying AI within products and services has implications well beyond the technical considerations, which often include change management of operational workflows or staffing levels, data privacy considerations, bias risks and other ethical implications, and industry-specific regulations on the use of data and models operationally. ', 't research in the field. AIPI 561: Operationalizing AI (MLOps) Deploying AI in production requires consideration of factors such as online model training, scaling, integration with software/hardware products, monitoring/support, security and failure resiliency. This course introduces stu

In [36]:
# prompt: I have a json file with 'input' and 'output' and want to add the result of semantic_search(input) as well as change the names of 'input' and 'output' to 'query' and 'answer'

for qa_pair in qa_pairs_json:
  query, _ = semantic_search(qa_pair['input'])
  qa_pair['context'] = query
  qa_pair['query'] = qa_pair['input']
  qa_pair['answer'] = qa_pair['output']
  del qa_pair['input']
  del qa_pair['output']

with open('updated_qa_pairs_0411.json', 'w') as f:
  json.dump(qa_pairs_json, f)


In [ ]:

from google.colab import files

files.download('updated_qa_pairs.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
updated_qa_pairs.json